# Librerias

In [ ]:
!pip uninstall -y pycaret scipy yellowbrick -q
!pip install scipy==1.11.4 -q
!pip install yellowbrick>=1.4 -q
!pip install pycaret -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 998.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 39.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xgboost 2.1.0 requires nvidia-nccl-cu12; platform_system == "Linux" and platform_machine != "aarch64", which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [ ]:
import sklearn
import pycaret
import pandas as pd
pd.set_option('display.max_columns', None)

print(f'scikit-learn version: {sklearn.__version__}')
print(f'pycaret version: {pycaret.__version__}')

scikit-learn version: 1.4.2
pycaret version: 3.3.2


In [ ]:
df = pd.read_csv('/content/Muertes_Malaria_Brazil_ML.csv', encoding='latin1')
display(df.head(3))

,COD_MUNICIPIO_RES_6,COD_MUNICIPIO_RES_7,ESTADO_RESIDENCIA,NOMBRE_MUNICIPIO_RESIDENCIA,COD_MUNICIPIO_OCURRENCIA,ANIO_REGISTRO,MES_REGISTRO,SEXO,RAZA_COLOR,ESCOLARIDAD,RANGO_EDAD,LATITUD,LONGITUD,CASOS_TOTALES,ANIO_MES,TEMPERATURA,HUMEDAD_RELATIVA,CATEGORIA_TEMPERATURA,CATEGORIA_HUMEDAD,CAUSA_BASICA
0,110006,1100064,Rondonia,Colorado do oeste,1100205,2004,9,M,Blanca,Sin especificar,65+,-13.11740,-60.5454,1,2004.750000,29.464,47.667,Calido,Muy baja,Malaria falciparum
1,150580,1505809,Para,Portel,150580,2006,10,F,Parda,Sin especificar,0 a 4,-1.93639,-50.8194,1,2006.833333,31.279,56.645,Muy calido,Baja,Malaria falciparum
2,150670,1506708,Para,Santana do araguaia,1506708,2003,4,M,Parda,Ninguna,35 a 39,-9.32810,-50.3500,1,2003.333333,24.707,93.933,Calido,Alta,Malaria falciparum


In [ ]:
df['SEXO'] = df['SEXO'].replace('I', 'F')

In [ ]:
df['SEXO'].value_counts()

,count
SEXO,
M,597
F,393


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   COD_MUNICIPIO_RES_6          990 non-null    int64  
 1   COD_MUNICIPIO_RES_7          990 non-null    int64  
 2   ESTADO_RESIDENCIA            990 non-null    object 
 3   NOMBRE_MUNICIPIO_RESIDENCIA  990 non-null    object 
 4   COD_MUNICIPIO_OCURRENCIA     990 non-null    int64  
 5   ANIO_REGISTRO                990 non-null    int64  
 6   MES_REGISTRO                 990 non-null    int64  
 7   SEXO                         990 non-null    object 
 8   RAZA_COLOR                   990 non-null    object 
 9   ESCOLARIDAD                  990 non-null    object 
 10  RANGO_EDAD                   990 non-null    object 
 11  LATITUD                      990 non-null    float64
 12  LONGITUD                     990 non-null    float64
 13  CASOS_TOTALES       

# Codificacion de Columnas

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Asegurar que los índices sean únicos
df_ml = df.copy()  # Hacer una copia del DataFrame original para trabajar
df_ml = df_ml.reset_index(drop=True)

columnas_a_eliminar = ['ANIO_MES','ESTADO_RESIDENCIA', 'NOMBRE_MUNICIPIO_RESIDENCIA','TEMPERATURA','HUMEDAD_RELATIVA','CATEGORIA_TEMPERATURA','CATEGORIA_HUMEDAD']
df_ml = df_ml.drop(columns=columnas_a_eliminar)
df_ml = df_ml.reset_index(drop=True)

# Definir las columnas categóricas a codificar
# data_to_label = ['SEXO', 'RAZA_COLOR', 'ESCOLARIDAD', 'RANGO_EDAD','CATEGORIA_TEMPERATURA','CATEGORIA_HUMEDAD']
data_to_label = ['SEXO', 'RAZA_COLOR', 'ESCOLARIDAD', 'RANGO_EDAD']

# Crear el codificador LabelEncoder para la etiqueta
label_encoder = LabelEncoder()

# Codificar la columna 'CAUSA_BASICA' (etiqueta)
df_ml['CAUSA_BASICA'] = label_encoder.fit_transform(df_ml['CAUSA_BASICA'])

# Separar la columna 'CAUSA_BASICA' (etiqueta) del resto del DataFrame
y = df_ml.pop('CAUSA_BASICA')

# Crear el codificador OneHotEncoder para las características categóricas
encoder = OneHotEncoder(sparse_output=False, drop='first')  # drop='first' para evitar colinearidad
encoded_array = encoder.fit_transform(df_ml[data_to_label])

# Obtener los nombres de las nuevas columnas
new_column_names = encoder.get_feature_names_out(data_to_label)

# Convertir la matriz a un DataFrame
df_encoded = pd.DataFrame(encoded_array, columns=new_column_names)

# Asegurar que los índices sean únicos y alineados
df_encoded.index = df_ml.index

# Concatenar con el DataFrame original sin las columnas codificadas
X = pd.concat([df_ml.drop(data_to_label, axis=1), df_encoded], axis=1)


# Random Forest Classifier

In [ ]:

# Ahora X contiene las características y y contiene la etiqueta codificada como números
# Puedes usar X y y para entrenar tu modelo de clasificación

# Ejemplo de entrenamiento de modelo de clasificación
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear el modelo de clasificación
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo
clf.fit(X_train, y_train)

# Hacer predicciones
y_pred = clf.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 45.45%


# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear el modelo de clasificación KNN
clf = KNeighborsClassifier(n_neighbors=5)  # Aquí puedes ajustar el número de vecinos (k)

# Entrenar el modelo
clf.fit(X_train, y_train)

# Hacer predicciones
y_pred = clf.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 42.09%


# AutoML

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pycaret
from pycaret.classification import *

# Asegurar que los índices sean únicos
df_ml = df.copy()  # Hacer una copia del DataFrame original para trabajar
df_ml = df_ml.reset_index(drop=True)

columnas_a_eliminar = ['ANIO_MES','ESTADO_RESIDENCIA', 'NOMBRE_MUNICIPIO_RESIDENCIA','TEMPERATURA','HUMEDAD_RELATIVA','CATEGORIA_TEMPERATURA','CATEGORIA_HUMEDAD']
df_ml = df_ml.drop(columns=columnas_a_eliminar)
df_ml = df_ml.reset_index(drop=True)

# Definir las columnas categóricas a codificar
data_to_label = ['SEXO', 'RAZA_COLOR', 'ESCOLARIDAD', 'RANGO_EDAD']
# data_to_label = ['SEXO', 'RAZA_COLOR', 'ESCOLARIDAD', 'RANGO_EDAD','CATEGORIA_TEMPERATURA','CATEGORIA_HUMEDAD']

# Crear el codificador LabelEncoder para la etiqueta
label_encoder = LabelEncoder()

# Codificar la columna 'CAUSA_BASICA' (etiqueta)
df_ml['CAUSA_BASICA'] = label_encoder.fit_transform(df_ml['CAUSA_BASICA'])

# Separar la columna 'CAUSA_BASICA' (etiqueta) del resto del DataFrame
y = df_ml.pop('CAUSA_BASICA')

# Crear el codificador OneHotEncoder para las características categóricas
encoder = OneHotEncoder(sparse_output=False, drop='first')  # drop='first' para evitar colinearidad
encoded_array = encoder.fit_transform(df_ml[data_to_label])

# Obtener los nombres de las nuevas columnas
new_column_names = encoder.get_feature_names_out(data_to_label)

# Convertir la matriz a un DataFrame
df_encoded = pd.DataFrame(encoded_array, columns=new_column_names)

# Asegurar que los índices sean únicos y alineados
df_encoded.index = df_ml.index

# Concatenar con el DataFrame original sin las columnas codificadas
X = pd.concat([df_ml.drop(data_to_label, axis=1), df_encoded], axis=1)

# Concatenar las características y la etiqueta en un solo DataFrame para PyCaret
df_pycaret = pd.concat([X, y], axis=1)

# Configurar el entorno de PyCaret
exp_clf = setup(data=df_pycaret, target='CAUSA_BASICA', train_size=0.7, session_id=42)

# Comparar modelos
best_model = compare_models()

# Afinar el mejor modelo
tuned_model = tune_model(best_model)

# Evaluar el modelo afinado
evaluate_model(tuned_model)

# Hacer predicciones con el modelo afinado
predictions = predict_model(tuned_model, data=df_pycaret)

# Mostrar las primeras filas de las predicciones
predictions.head()


,Description,Value
0,Session id,42
1,Target,CAUSA_BASICA
2,Target type,Multiclass
3,Original data shape,"(990, 34)"
4,Transformed data shape,"(990, 34)"
5,Transformed train set shape,"(693, 34)"
6,Transformed test set shape,"(297, 34)"
7,Numeric features,33
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,0.4691,0.3721,0.4691,0.4792,0.4619,0.2026,0.2072,0.0740
lr,Logistic Regression,0.4663,0.0000,0.4663,0.4625,0.4563,0.1967,0.2003,2.0180
rf,Random Forest Classifier,0.4590,0.3888,0.4590,0.4566,0.4529,0.1845,0.1867,0.3060
et,Extra Trees Classifier,0.4561,0.3688,0.4561,0.4556,0.4509,0.1826,0.1841,0.2560
xgboost,Extreme Gradient Boosting,0.4445,0.3757,0.4445,0.4443,0.4392,0.1628,0.1644,0.2850
lightgbm,Light Gradient Boosting Machine,0.4401,0.3834,0.4401,0.4412,0.4369,0.1586,0.1596,1.4250
gbc,Gradient Boosting Classifier,0.4344,0.0000,0.4344,0.4357,0.4310,0.1499,0.1513,1.0790
dt,Decision Tree Classifier,0.4301,0.3440,0.4301,0.4261,0.4261,0.1453,0.1461,0.1380
ridge,Ridge Classifier,0.4128,0.0000,0.4128,0.4136,0.4068,0.1124,0.1142,0.0450
lda,Linear Discriminant Analysis,0.4113,0.0000,0.4113,0.4138,0.4078,0.1154,0.1169,0.0390


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5286,0.6725,0.5286,0.5273,0.5265,0.2955,0.2963
1,0.4571,0.6486,0.4571,0.4602,0.4524,0.1920,0.1945
2,0.3429,0.5250,0.3429,0.3451,0.3431,0.0213,0.0213
3,0.4493,0.0000,0.4493,0.4678,0.4515,0.1723,0.1752
4,0.6232,0.0000,0.6232,0.6231,0.6230,0.4310,0.4312
5,0.3913,0.0000,0.3913,0.3999,0.3942,0.0835,0.0838
6,0.5217,0.0000,0.5217,0.5238,0.5201,0.2758,0.2772
7,0.4493,0.5994,0.4493,0.4571,0.4470,0.1757,0.1782
8,0.4928,0.6238,0.4928,0.4864,0.4856,0.2326,0.2343


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,K Neighbors Classifier,0.8253,0.9636,0.8253,0.8264,0.8253,0.7387,0.7389


,COD_MUNICIPIO_RES_6,COD_MUNICIPIO_RES_7,COD_MUNICIPIO_OCURRENCIA,ANIO_REGISTRO,MES_REGISTRO,LATITUD,LONGITUD,CASOS_TOTALES,SEXO_M,RAZA_COLOR_Blanca,...,RANGO_EDAD_45 a 49,RANGO_EDAD_5 a 9,RANGO_EDAD_50 a 54,RANGO_EDAD_55 a 59,RANGO_EDAD_60 a 64,RANGO_EDAD_65+,RANGO_EDAD_Sin especificar,CAUSA_BASICA,prediction_label,prediction_score
0,110006,1100064,1100205,2004,9,-13.117400,-60.545399,1,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,1.0000
1,150580,1505809,150580,2006,10,-1.936390,-50.819401,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.4789
2,150670,1506708,1506708,2003,4,-9.328100,-50.349998,1,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.9928
3,530010,5300108,5300108,2004,8,-15.779500,-47.929699,1,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2,0.3920
4,160027,1600279,1600279,2005,1,-0.804911,-52.452999,1,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1.0000


# Probando nuevos registros

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Crear las nuevas entradas de datos
nuevas_entradas = [
     {
        'COD_MUNICIPIO_RES_6': 110002,
        'COD_MUNICIPIO_RES_7': 5218003,
        'COD_MUNICIPIO_OCURRENCIA': 110009,
        'ANIO_REGISTRO': 2005,
        'MES_REGISTRO': 6,
        'SEXO': 'F',
        'RAZA_COLOR': 'Blanca',
        'ESCOLARIDAD': 'Ninguna',
        'RANGO_EDAD': '0 a 4',
        'LATITUD': -7.099,
        'LONGITUD': -56.106,
        'CASOS_TOTALES': 1
    },
    {
        'COD_MUNICIPIO_RES_6': 110094,
        'COD_MUNICIPIO_RES_7': 5300108,
        'COD_MUNICIPIO_OCURRENCIA': 1100064,
        'ANIO_REGISTRO': 2006,
        'MES_REGISTRO': 5,
        'SEXO': 'M',
        'RAZA_COLOR': 'Indigena',
        'ESCOLARIDAD': '1 a 3',
        'RANGO_EDAD': '25 a 29',
        'LATITUD': -11.099,
        'LONGITUD': -44.36,
        'CASOS_TOTALES': 1
    }
    ,
    {
        'COD_MUNICIPIO_RES_6': 67890,
        'COD_MUNICIPIO_RES_7': 1100064,
        'COD_MUNICIPIO_OCURRENCIA': 1100106,
        'ANIO_REGISTRO': 2003,
        'MES_REGISTRO': 12,
        'SEXO': 'M',
        'RAZA_COLOR': 'Negra',
        'ESCOLARIDAD': '4 a 7',
        'RANGO_EDAD': '40 a 44',
        'LATITUD': -8.099,
        'LONGITUD': -54.36,
        'CASOS_TOTALES': 1
    }
    ,
    {
        'COD_MUNICIPIO_RES_6': 530010,
        'COD_MUNICIPIO_RES_7': 1100031,
        'COD_MUNICIPIO_OCURRENCIA': 120001,
        'ANIO_REGISTRO': 2007,
        'MES_REGISTRO': 5,
        'SEXO': 'F',
        'RAZA_COLOR': 'Parda',
        'ESCOLARIDAD': '8 a 11',
        'RANGO_EDAD': '60 a 64',
        'LATITUD': -11.328,
        'LONGITUD': -52.350,
        'CASOS_TOTALES': 1
    }
    ,
    {
        'COD_MUNICIPIO_RES_6': 530019,
        'COD_MUNICIPIO_RES_7': 1100033,
        'COD_MUNICIPIO_OCURRENCIA': 120005,
        'ANIO_REGISTRO': 2005,
        'MES_REGISTRO': 9,
        'SEXO': 'M',
        'RAZA_COLOR': 'Blanca',
        'ESCOLARIDAD': 'Ninguna',
        'RANGO_EDAD': '10 a 14',
        'LATITUD': -14.928,
        'LONGITUD': -42.750,
        'CASOS_TOTALES': 1
    }
]

df_nuevas_entradas = pd.DataFrame(nuevas_entradas)

# Aplicar la misma transformación OneHotEncoder a las nuevas entradas
encoded_array_nuevas_entradas = encoder.transform(df_nuevas_entradas[data_to_label])
df_encoded_nuevas_entradas = pd.DataFrame(encoded_array_nuevas_entradas, columns=new_column_names)
df_encoded_nuevas_entradas.index = df_nuevas_entradas.index

# Concatenar con el DataFrame original sin las columnas codificadas
X_nuevas_entradas = pd.concat([df_nuevas_entradas.drop(data_to_label, axis=1), df_encoded_nuevas_entradas], axis=1)

# Hacer predicciones con el modelo afinado para cada entrada individualmente
resultados = []

for i in range(X_nuevas_entradas.shape[0]):
    prediccion = predict_model(tuned_model, data=X_nuevas_entradas.iloc[[i]])
    response_score = prediccion['prediction_score'].values[0] if 'prediction_score' in prediccion.columns else prediccion['Score'].values[0]
    response_value = prediccion['prediction_label'].values[0] if 'prediction_label' in prediccion.columns else prediccion['Label'].values[0]
    tipo_de_malaria = label_encoder.inverse_transform([response_value])[0]
    prediction_info = f"Etiqueta '{response_value}' tipo de malaria: {tipo_de_malaria} ({response_score * 100:.2f}%)"

    resultado = df_nuevas_entradas.iloc[i].copy()
    resultado['CAUSA_BASICA'] = tipo_de_malaria
    resultado['prediction_label'] = response_value
    resultado['prediction_score'] = response_score * 100
    resultado['prediction_info'] = prediction_info

    resultados.append(resultado)

# Crear un DataFrame con los resultados
df_resultados = pd.DataFrame(resultados)

# Configurar pandas para mostrar todas las columnas sin truncar
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Mostrar los resultados
display(df_resultados)


,COD_MUNICIPIO_RES_6,COD_MUNICIPIO_RES_7,COD_MUNICIPIO_OCURRENCIA,ANIO_REGISTRO,MES_REGISTRO,SEXO,RAZA_COLOR,ESCOLARIDAD,RANGO_EDAD,LATITUD,LONGITUD,CASOS_TOTALES,CAUSA_BASICA,prediction_label,prediction_score,prediction_info
0,110002,5218003,110009,2005,6,F,Blanca,Ninguna,0 a 4,-7.099,-56.106,1,Sin especificar,3,55.78,Etiqueta '3' tipo de malaria: Sin especificar (55.78%)
1,110094,5300108,1100064,2006,5,M,Indigena,1 a 3,25 a 29,-11.099,-44.360,1,Sin especificar,3,55.25,Etiqueta '3' tipo de malaria: Sin especificar (55.25%)
2,67890,1100064,1100106,2003,12,M,Negra,4 a 7,40 a 44,-8.099,-54.360,1,Malaria falciparum,0,53.18,Etiqueta '0' tipo de malaria: Malaria falciparum (53.18%)
3,530010,1100031,120001,2007,5,F,Parda,8 a 11,60 a 64,-11.328,-52.350,1,Malaria vivax,2,61.11,Etiqueta '2' tipo de malaria: Malaria vivax (61.11%)
4,530019,1100033,120005,2005,9,M,Blanca,Ninguna,10 a 14,-14.928,-42.750,1,Malaria vivax,2,63.89,Etiqueta '2' tipo de malaria: Malaria vivax (63.89%)


# Mapa Interactivo Data real vs Data nueva

In [ ]:
import folium
from folium.plugins import HeatMap

# Crear un mapa centrado en el promedio de latitud y longitud de las entradas existentes
mapa_calor = folium.Map(location=[df_resultados['LATITUD'].mean(), df_resultados['LONGITUD'].mean()], zoom_start=5)

# Datos para el mapa de calor
puntos = list(zip(df_resultados['LATITUD'], df_resultados['LONGITUD']))
HeatMap(puntos).add_to(mapa_calor)

# Función para obtener datos del tooltip para df_resultados
def get_tooltip_data_resultados(index):
    row = df_resultados.iloc[index]
    tooltip = f'''
        <h4>Información adicional:</h4>
        <p><strong>Municipio:</strong> {row.get('NOMBRE_MUNICIPIO_RESIDENCIA', 'N/A')}</p>
        <p><strong>Causa básica:</strong> {row.get('CAUSA_BASICA', 'N/A')}</p>
        <p><strong>Rango de edad:</strong> {row.get('RANGO_EDAD', 'N/A')}</p>
        <p><strong>Predicción:</strong> {row.get('prediction_info', 'N/A')}</p>
    '''
    return tooltip

# Añadir marcadores para los datos existentes
for i in range(len(df)):
    tooltip = get_tooltip_data(i)
    folium.Marker(
        [df.loc[i, 'LATITUD'], df.loc[i, 'LONGITUD']],
        tooltip=tooltip
    ).add_to(mapa_calor)

# Añadir marcadores para los datos de df_resultados
for i in range(len(df_resultados)):
    tooltip = get_tooltip_data_resultados(i)
    folium.Marker(
        [df_resultados.loc[i, 'LATITUD'], df_resultados.loc[i, 'LONGITUD']],
        tooltip=tooltip
    ).add_to(mapa_calor)

# Añadir marcadores para nuevas entradas en color rojo (ya incluidas en df_resultados)
for i in range(len(df_resultados)):
    folium.Marker(
        location=[df_resultados.loc[i, 'LATITUD'], df_resultados.loc[i, 'LONGITUD']],
        popup=f'''
            <h4>Información nueva:</h4>
            <p><strong>Municipio de Residencia:</strong> {df_resultados.loc[i, 'COD_MUNICIPIO_RES_6']}</p>
            <p><strong>Causa Básica:</strong> {df_resultados.loc[i, 'CAUSA_BASICA']}</p>
            <p><strong>Rango de Edad:</strong> {df_resultados.loc[i, 'RANGO_EDAD']}</p>
            <p><strong>Predicción:</strong> {df_resultados.loc[i, 'prediction_info']}</p>
        ''',
        icon=folium.Icon(color='red', icon='info-sign')  # Icono de color rojo
    ).add_to(mapa_calor)

# Estadísticas resumidas para df_resultados
total_casos = len(df_resultados)
casos_por_estado = df_resultados['ESTADO_RESIDENCIA'].value_counts() if 'ESTADO_RESIDENCIA' in df_resultados.columns else 'No disponible'

estadisticas_html = f'''
    <div style="position: fixed; top: 10px; right: 10px; z-index:1000; background-color:white; padding: 10px; border: 2px solid black; border-radius: 5px;">
        <h4>Estadísticas resumidas:</h4>
        <p><strong>Total de casos:</strong> {total_casos}</p>
        <h5>Casos por estado:</h5>
        <ul>
    '''
if casos_por_estado != 'No disponible':
    for estado, casos in casos_por_estado.items():
        estadisticas_html += f'<li>{estado}: {casos}</li>'
estadisticas_html += '</ul></div>'

mapa_calor.get_root().html.add_child(folium.Element(estadisticas_html))

# Añadir controles de capa y posición del mouse
folium.LayerControl().add_to(mapa_calor)
folium.plugins.MousePosition().add_to(mapa_calor)

# Mostrar el mapa
mapa_calor
